## Example: Developing a Bernoulli Binary Bandit Ticker Picker Agent
In this example, we'll develop a collection of agents to advise us which stocks we should include in a portfolio $\mathcal{P}$ by solving a Multi-arm Binary Bandit problem using [ϵ-Greedy Thompson Sampling](https://arxiv.org/abs/1707.02038). The `K`-actions (bandits) are the potential `tickers` in the portfolio $\mathcal{P}$ portfolio. After their analysis, we'll ask the agents to rank-order their preferred `tickers`. 

### Problem
* We have `N` agents independently analyzing daily Open High Low Close (OHLC) data sequences and rank-ordering their belief that the ticker `XYZ` will return at least the risk-free rate in the next step. 
* We then sample the `world`. If the ticker `XYZ` returns greater than or equal to the risk-free rate in the sample period, the agent receives a reward of `+1`. Otherwise, the agent receives a reward of `0`.
* Each agent develops a distribution of beliefs for the probability $p_{a}$, i.e., the probability that `ticker` will beat the risk-free rate based on this experimentation. These beliefs are encoded in the parameters of a [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution)

## Setup

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `C:\Users\Lillian K\OneDrive\Documents\CHEME 5760\5760-Final-Project-check\CHEME-5760-Final-Project\Project.toml`
  No Changes to `C:\Users\Lillian K\OneDrive\Documents\CHEME 5760\5760-Final-Project-check\CHEME-5760-Final-Project\Manifest.toml`
Precompiling project...
  ✓ GPUArraysCore
  ✓ CEnum
  ✓ BangBang → BangBangStaticArraysExt
  ✓ ArrayInterface → ArrayInterfaceGPUArraysCoreExt
  ✓ StructArrays
  ✓ FiniteDiff
  ✓ BangBang → BangBangStructArraysExt
  ✓ NLSolversBase
  ✓ FiniteDiff → FiniteDiffStaticArraysExt
  ✓ MicroCollections
  ✓ LsqFit
  ✓ LLVM
  ✓ Transducers
  ✓ ChainRules
  ✓ GPUArrays
  ✓ UnsafeAtomicsLLVM
  ✓ Transducers → TransducersDataFramesExt
  ✓ KernelAbstractions
  ✓ FLoops
  ✓ NNlib
  ✓ VLQuantitativeFinancePackage
  ✓ OneHotArrays
  ✓ MLUtils
  ✓ Zygote
  ✓ Zygote → ZygoteColorsExt
  ✓ Zygote → ZygoteDistancesExt
  ✓ Flux
  27

In [2]:
include(joinpath(_PATH_TO_SRC, "CHEME-5660-L14a-BanditProblems-CodeLibrary.jl"));

## Prerequisites: Load and clean the historical dataset
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) since `01-03-2018` until `11-17-2023`, along with data for a few exchange traded funds and volatility products during that time. 

In [3]:
original_dataset = load(joinpath(_PATH_TO_DATA, 
        "SP500-Daily-OHLC-1-3-2018-to-11-17-2023.jld2")) |> x-> x["dataset"];

### Clean the data
Not all of the tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquistion or de-listing events. Let's collect only those tickers with the maximum number of trading days.

* First, let's compute the number of records for a company that we know has a maximim value, e.g., `AAPL` and save that value in the `maximum_number_trading_days` variable:

In [4]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow

1480

Now, lets iterate through our data and collect only those tickers that have `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [5]:
dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
dataset;

Let's get a list of firms that we have in cleaned up `dataset`, and save it in the `all_tickers` array:

In [6]:
all_tickers = keys(dataset) |> collect |> sort;
K = length(all_tickers)

459

## Initialize the `world` function
The world function takes action `a`, i.e., chooses a `ticker` from the collection of stocks/ETFs that we are exploring, and asks whether or not the return of this `ticker` at the end of today was greater or equal to the risk-free rate. 
* If the daily return is greater than or equal to the risk-free rate, the `agent` receives a reward of `r = 1`. Otherwise, the `agent` receives a `r = 0` reward.

In [7]:
function world(action::Int64, start::Int64, data::Dict{String, DataFrame}, tickers::Array{String,1}; 
        Δt::Float64 = (1.0/252.0), risk_free_rate::Float64 = 0.05, buffersize::Int64 = 1)::Int64

    # initialize -
    result_flag = 0;

    # daily risk free rate -
    T = buffersize*Δt;

    # grab the ticker we are looking at?
    ticker_symbol = tickers[action];
    
    # compute the expected return over the horizon -
    price_df = data[ticker_symbol];
    time = range(start+1,(start+buffersize), step=1) |> collect
    buffer = Array{Float64,1}();
    for t ∈ time
        P₁ = price_df[t-1,  :volume_weighted_average_price]
        P₂ = price_df[t, :volume_weighted_average_price]
        R = (1/Δt)*log(P₂/P₁);
        push!(buffer,R);
    end
    μ = mean(buffer);

    # if we invested 1 USD, in each how much would we have at the end of horizon -
    W_risk_free = exp(risk_free_rate*T);
    W_ticker = exp(μ*T);
    
    # are we better or worse relative to the risk-free investment?
    if (W_ticker >= W_risk_free)
        result_flag = 1;
    end

    # default -
    return result_flag;
end;

Set the `risk_free_rate` variable:

In [8]:
risk_free_rate = 0.047;

## Setup the `ticker-picker` agent

First, let's specify the tickers that we want to examine in the `tickers` array, and store the number of tickers in the `K` variable:

In [9]:
all_tickers

459-element Vector{String}:
 "A"
 "AAL"
 "AAP"
 "AAPL"
 "ABBV"
 "ABT"
 "ACN"
 "ADBE"
 "ADI"
 "ADM"
 "ADP"
 "ADSK"
 "AEE"
 ⋮
 "WU"
 "WY"
 "WYNN"
 "XEL"
 "XOM"
 "XRAY"
 "XYL"
 "YUM"
 "ZBH"
 "ZBRA"
 "ZION"
 "ZTS"

In [12]:
tickers = all_tickers;
K = length(tickers);

Next, we construct the `EpsilonSamplingModel` instance which holds information about the ϵ-greedy sampling approach. The `EpsilonSamplingModel` type has one additional field, the `ϵ` field which controls the approximate fraction of `exploration` steps the algorithm takes; `exploration` steps are purely random.

In [13]:
model = EpsilonSamplingModel()
model.K = K; # tickers
model.α = ones(K); # initialize to uniform values
model.β = ones(K); # initialize to uniform values
model.ϵ = 0.3;

## Run a single `ticker-picker` agent and explore its preferences
Let's run a single `ticker-picker` agent and examine what it returns using the `sample(...)` function. 
* The `sample(...)` function takes the agent model `model::EpsilonSamplingModel`, the `world::Function`, the cleaned `dataset` and your list of `tickers`, along with the `horizon` parameter, i.e., how many iterations we want the search to run for,  and the `risk_free_rate`. 
* The `sample(...)` function returns a dictionary holding the $(\alpha,\beta)$ parameters for each ticker (values) for iteration (keys).  

In [14]:
time_sample_results_dict_eps = sample(model, world, dataset, tickers; 
    horizon = (maximum_number_trading_days - 1), risk_free_rate = risk_free_rate)

Dict{Int64, Matrix{Float64}} with 1479 entries:
  1144 => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 2.0 2.0]
  1175 => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 2.0 2.0]
  719  => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 1.0 1.0]
  1028 => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 2.0 1.0]
  699  => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 1.0 1.0]
  831  => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 1.0 1.0]
  1299 => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 2.0 2.0]
  1438 => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 2.0 2.0]
  1074 => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 2.0 2.0]
  319  => [1.0 2.0; 2.0 1.0; … ; 2.0 1.0; 1.0 1.0]
  687  => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 1.0 1.0]
  1199 => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 2.0 2.0]
  185  => [1.0 2.0; 2.0 1.0; … ; 1.0 1.0; 1.0 1.0]
  823  => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 1.0 1.0]
  1090 => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 2.0 2.0]
  420  => [1.0 2.0; 2.0 1.0; … ; 2.0 1.0; 1.0 1.0]
  1370 => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 2.0 2.0]
  1437 => [1.0 2.0; 2.0 2.0; … ; 2.0 2.0; 2.0 2.0]
  525  => [1.0 2.0; 2.0 2.0; … ; 2

In [15]:
Z = time_sample_results_dict_eps[1479]

459×2 Matrix{Float64}:
 1.0  2.0
 2.0  2.0
 2.0  2.0
 1.0  2.0
 2.0  3.0
 3.0  2.0
 1.0  2.0
 1.0  2.0
 1.0  1.0
 1.0  2.0
 1.0  2.0
 1.0  2.0
 3.0  2.0
 ⋮    
 1.0  2.0
 1.0  2.0
 1.0  2.0
 1.0  2.0
 6.0  3.0
 3.0  2.0
 1.0  2.0
 1.0  2.0
 2.0  3.0
 1.0  2.0
 2.0  2.0
 2.0  2.0

## Run a collection of `ticker-picker` agents and examine their preferences
Repeat the single-agent analysis with `N` agents by running the `sample(...)` method inside a `for` loop. We'll store the results of the last time point in the `agent_specific_data::Array{Beta,2}(undef, K, number_of_agents)` array. 
* The `agent_specific_data` array holds the `Beta` distributions for each agent, i.e., it holds the preferences for each agent (cols) for each `ticker` in our collection (rows).

In [ ]:
number_of_agents = 10000;
trading_day_index = 1479
agent_specific_data = Array{Beta,2}(undef, K, number_of_agents);

for agent_index ∈ 1:number_of_agents
    
    # sample -
    time_sample_results_dict_eps = sample(model, world, dataset, tickers; horizon = (maximum_number_trading_days - 1), risk_free_rate = risk_free_rate);
    beta_array = build_beta_array(time_sample_results_dict_eps[trading_day_index]);

    # grab data for this agent -
    for k = 1:K
        agent_specific_data[k, agent_index] = beta_array[k]
    end
end

In [14]:
agent_specific_data[1,:] # data for each agent for ticker 1 for all the agents

10000-element Vector{Beta}:
 Beta{Float64}(α=16.0, β=15.0)
 Beta{Float64}(α=30.0, β=24.0)
 Beta{Float64}(α=29.0, β=29.0)
 Beta{Float64}(α=22.0, β=24.0)
 Beta{Float64}(α=40.0, β=32.0)
 Beta{Float64}(α=4.0, β=7.0)
 Beta{Float64}(α=3.0, β=7.0)
 Beta{Float64}(α=6.0, β=11.0)
 Beta{Float64}(α=3.0, β=8.0)
 Beta{Float64}(α=7.0, β=12.0)
 Beta{Float64}(α=17.0, β=16.0)
 Beta{Float64}(α=30.0, β=29.0)
 Beta{Float64}(α=38.0, β=34.0)
 ⋮
 Beta{Float64}(α=22.0, β=18.0)
 Beta{Float64}(α=3.0, β=9.0)
 Beta{Float64}(α=42.0, β=31.0)
 Beta{Float64}(α=8.0, β=11.0)
 Beta{Float64}(α=11.0, β=15.0)
 Beta{Float64}(α=21.0, β=16.0)
 Beta{Float64}(α=69.0, β=52.0)
 Beta{Float64}(α=26.0, β=25.0)
 Beta{Float64}(α=2.0, β=6.0)
 Beta{Float64}(α=20.0, β=20.0)
 Beta{Float64}(α=7.0, β=10.0)
 Beta{Float64}(α=6.0, β=11.0)

## The wisdom of the collective
Now that we have prefernces for the `N` agents (encoded as `Beta` distributions for each ticker), let's develop a concencous belief in which tickers to include in our portfolio $\mathcal{P}$. First, let's compute the agent-specific rank of eack ticker, where `rank = 1` is the best, and `rank = K` is the worst. We'll store these values in the `preference_rank_array` array.

In [15]:
preference_rank_array = Array{Int,2}(undef, number_of_agents, K);
for agent ∈ 1:number_of_agents
        
    # ask agent about thier preference for ticker i -
    experience_distributions = agent_specific_data[:,agent]
    preference_vector = preference(experience_distributions, tickers) .|> x-> trunc(Int64,x) # trunc function is cool!
    
    # package -
    for i ∈ 1:K
        preference_rank_array[agent, i] = preference_vector[i];
    end
end
preference_rank_array

10000×24 Matrix{Int64}:
 11   4   7  19   3   1  17   5   2  …  17  15  23  13  13  22  16  13  21
  7   3  15  13   9  21   4  18   2      1  19   6  11   7  12  23  10  20
 10   4  22   2  21  24   6   8   1     15   9  13  12  19   3  10  14  23
 15  11  17   4  22   2   3  20  24     19   9  23  10   4   6   8  16   1
  7   4  11  14   8   5  13  21  17     22  10   1   9   3  19  23   2  20
 21  22  19  18  13   7  15  19   2  …   7   9  14  24  11   6  12  16   1
 21  20   5   4  19  22   3  12   8     16  23  13  15  18  23  10  16   2
 21  18  14  11   4  16  15   6   9     19   1  20   9  23   2  12  22   3
 23  20  14  13   7  10   8   5   6     12  16  15  16   1   3   2  22  19
 20   8   7  14   1   5  24   3  17      4  13  22  11  16  12  21  23   6
  7  12  18   9  22   5  14   4   6  …  10   1  19  21  20  17  23  24  16
 12  10  17  16   1   9  11   3   6     23   7  20  21  15   8   2   5  19
  5  13   2  10  10  23   3  22  21      6  17  19  19  12   1   9  14  16
 

### Compute the frequency dictionary
Let's count the times a `ticker` is ranked in the `top 3` across the `N` agents and then normalize by the number of agents, i.e., compute the frequency of being ranked in the `top 3`. We'll store this value in the `frequency_dictionary,` where the `ticker` is the key, and the frequency is the value.
* `Hypothesis`: Those `tickers` with high rank are likelier to beat an alternative `risk-free` investment, while low rank `tickers` do not outperform a `risk-free` alternative investment. 

In [22]:
frequency_dictionary = Dict{String,Float64}();
for i ∈ eachindex(tickers)
    
    # compute the frequency -
    freq = findall(x-> x ≤ 8, preference_rank_array[:,i]) |> x-> length(x) |> x-> x/number_of_agents
    
    # get the ticker -
    ticker = tickers[i];
    frequency_dictionary[ticker] = freq;
end

In [23]:
frequency_dictionary

Dict{String, Float64} with 24 entries:
  "MSFT"  => 0.5366
  "JPM"   => 0.3625
  "C"     => 0.2212
  "MRK"   => 0.3284
  "UNH"   => 0.2926
  "SPY"   => 0.4712
  "BAC"   => 0.3175
  "MU"    => 0.2603
  "SRE"   => 0.3832
  "TSLA"  => 0.3932
  "AMD"   => 0.3925
  "INTC"  => 0.2928
  "MMM"   => 0.2243
  "ECL"   => 0.3067
  "LMT"   => 0.2588
  "OXY"   => 0.2719
  "CMCSA" => 0.2493
  "JNJ"   => 0.2838
  "SPYD"  => 0.3198
  "NFLX"  => 0.3004
  "WFC"   => 0.3367
  "PEP"   => 0.3649
  "MET"   => 0.4227
  "AAPL"  => 0.4542

### Disclaimer and Risks
__This content is offered solely for training and  informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy,  is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on your evaluation of your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.